In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from sklearn.preprocessing import StandardScaler

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_X = pd.read_csv("../data/train_X_Scaled.csv")
test_X = pd.read_csv("../data/test_X_Scaled.csv")
train_y = np.loadtxt("../data/train_y.csv")

In [3]:
test_X.head()

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,item_seq_number,user_type,...,param_1_scale,param_2_scale,param_3_scale,price_new,image_new,price_scaled,item_seq_number_scaled,image_top_1_scaled,title_nwords_scaled,desc_nwords_scaled
0,22,996,4,2,14,110,989,NaN,66,1,...,-1.045182,0.889876,3.392737,279818.945942,2020.000000,1.119299e-17,-0.134434,7.849013e-01,-0.739803,-0.608671
1,11,1745,8,42,23,16,244,3000.0,4,1,...,-0.959405,-0.718366,-0.390792,3000.000000,1297.959228,-5.323064e-02,-0.145414,-2.471687e-16,-0.739803,-0.415612
2,4,569,0,12,244,16,244,15000.0,15,1,...,1.146887,-0.718366,-0.390792,15000.000000,2960.000000,-5.092311e-02,-0.143466,1.806737e+00,-1.280793,-0.150155
3,10,357,2,34,11,37,244,4500.0,70,1,...,-1.073774,-0.359078,-0.390792,4500.000000,1297.959228,-5.294220e-02,-0.133726,-2.471687e-16,-0.198814,-0.367347
4,6,913,4,37,5,16,244,4900.0,15,1,...,-1.130958,-0.718366,-0.390792,4900.000000,1002.000000,-5.286528e-02,-0.143466,-3.217253e-01,-0.739803,-0.415612


In [5]:
train_X = train_X[['param_1_scaled', 'param_2_scaled','param_3_scaled',\
                  'price_scaled', 'item_seq_number_scaled', 'image_top_1_scaled',\
                  'title_nwords_scaled', 'desc_nwords_scaled', 'user_type']]
test_X = test_X[['param_1_scale', 'param_2_scale','param_3_scale',\
                  'price_scaled', 'item_seq_number_scaled', 'image_top_1_scaled',\
                  'title_nwords_scaled', 'desc_nwords_scaled', 'user_type']]

In [19]:
train_X[199990:200000]

,param_1_scaled,param_2_scaled,param_3_scaled,price_scaled,item_seq_number_scaled,image_top_1_scaled,title_nwords_scaled,desc_nwords_scaled,user_type
199990,-1.049005,-0.457388,0.241832,-0.004860,-0.129326,-1.225529e+00,-0.717567,-0.502101,1
199991,0.035633,0.886540,-0.015622,-0.004848,-0.119277,-8.312797e-01,-0.176178,-0.452388,1
199992,-0.828238,1.093298,0.014667,-0.004863,-0.132915,-9.255568e-01,-0.717567,-0.502101,1
199993,-1.096998,-0.733066,-0.409375,-0.004814,-0.130941,-4.488147e-01,-1.258955,-0.303249,1
199994,0.985891,0.800391,-0.409375,-0.004752,-0.133274,-1.217962e-15,-0.717567,-0.402675,1
199995,-1.049005,1.093298,3.684649,-0.004867,-0.128788,-1.242671e+00,-1.258955,-0.352962,1
199996,-1.049005,-0.457388,0.241832,-0.004869,-0.115867,-1.240528e+00,-0.176178,0.169025,1
199997,-0.741851,-0.733066,-0.409375,-0.004873,-0.132915,-4.220315e-01,-0.717567,-0.402675,1
199998,0.035633,2.040940,0.443757,-0.004867,-0.117124,-6.191563e-01,-0.717567,-0.352962,1
199999,0.985891,-0.147251,-0.187258,-0.004721,-0.133095,-1.217962e-15,-0.717567,-0.452388,1


In [6]:
# train_X["price_new"] = train_X["price"].values
# train_X["price_new"].fillna(np.nanmean(train_X["price"].values), inplace=True)

# train_X["image_new"] = train_X["image_top_1"].values
# train_X["image_new"].fillna(np.nanmean(train_X["image_top_1"].values), inplace=True)

In [7]:
# #数值类数据标准化
# def fun_scale(df_feature):
#     np_feature = df_feature.values.reshape(-1,1).astype(np.float64)
#     feature_scale = StandardScaler().fit(np_feature)
#     feature_scaled = StandardScaler().fit_transform(np_feature, feature_scale)
#     return feature_scale, feature_scaled

In [8]:
# param_1_scale, train_X['param_1_scaled'] = fun_scale(train_X['param_1'])
# param_2_scale, train_X['param_2_scaled'] = fun_scale(train_X['param_2'])
# param_3_scale, train_X['param_3_scaled'] = fun_scale(train_X['param_3'])
# price_scale, train_X['price_scaled'] = fun_scale(train_X['price_new'])
# item_seq_number_scale, train_X['item_seq_number_scaled'] = fun_scale(train_X['item_seq_number'])
# image_top_1_scale, train_X['image_top_1_scaled'] = fun_scale(train_X['image_new'])
# title_nwords_scale, train_X['title_nwords_scaled'] = fun_scale(train_X['title_nwords'])
# desc_nwords_scale, train_X['desc_nwords_scaled'] = fun_scale(train_X['desc_nwords'])

In [9]:
# def fun_test_scale(feature_scale, df_feature):
#     np_feature = df_feature.values.reshape(-1,1).astype(np.float64)
#     feature_scaled = StandardScaler().fit_transform(np_feature, feature_scale)
#     return feature_scaled



# test_X["price_new"] = test_X["price"].values
# test_X["price_new"].fillna(np.nanmean(test_X["price"].values), inplace=True)

# test_X["image_new"] = test_X["image_top_1"].values
# test_X["image_new"].fillna(np.nanmean(test_X["image_top_1"].values), inplace=True)


# test_X['param_1_scale'] = fun_test_scale(param_1_scale, test_X['param_1'])
# test_X['param_2_scale'] = fun_test_scale(param_2_scale, test_X['param_2'])
# test_X['param_3_scale'] = fun_test_scale(param_3_scale, test_X['param_3'])
# test_X['price_scaled'] = fun_test_scale(price_scale, test_X['price_new'])
# test_X['item_seq_number_scaled'] = fun_test_scale(item_seq_number_scale, test_X['item_seq_number'])
# test_X['image_top_1_scaled'] = fun_test_scale(image_top_1_scale, test_X['image_new'])
# test_X['title_nwords_scaled'] = fun_test_scale(title_nwords_scale, test_X['title_nwords'])
# test_X['desc_nwords_scaled'] = fun_test_scale(desc_nwords_scale, test_X['desc_nwords'])

In [10]:
test_X.head()

,param_1_scale,param_2_scale,param_3_scale,price_scaled,item_seq_number_scaled,image_top_1_scaled,title_nwords_scaled,desc_nwords_scaled,user_type
0,-1.045182,0.889876,3.392737,1.119299e-17,-0.134434,7.849013e-01,-0.739803,-0.608671,1
1,-0.959405,-0.718366,-0.390792,-5.323064e-02,-0.145414,-2.471687e-16,-0.739803,-0.415612,1
2,1.146887,-0.718366,-0.390792,-5.092311e-02,-0.143466,1.806737e+00,-1.280793,-0.150155,1
3,-1.073774,-0.359078,-0.390792,-5.294220e-02,-0.133726,-2.471687e-16,-0.198814,-0.367347,1
4,-1.130958,-0.718366,-0.390792,-5.286528e-02,-0.143466,-3.217253e-01,-0.739803,-0.415612,1


In [11]:
# test_X.to_csv("../data/test_X_scaled.csv",index=False,sep=',')
# train_X.to_csv("../data/train_X_scaled.csv",index=False,sep=',')

In [12]:
dev_X = train_X.iloc[:-200000,:].values
val_X = train_X.iloc[-200000:,:].values
dev_y = train_y[:-200000]
val_y = train_y[-200000:]
dev_y = dev_y.reshape([dev_y.shape[0], 1])
val_y = val_y.reshape([val_y.shape[0], 1])
print(dev_X.shape, val_X.shape, test_X.shape)
print(dev_y.shape, val_y.shape)

(1303424, 9) (200000, 9) (508438, 9)
(1303424, 1) (200000, 1)


In [13]:
def get_weight(shape, lambd): #lambd : parameter of regularization
    var = tf.Variable(tf.random_normal(shape, stddev=0.5,dtype=tf.float32), dtype=tf.float32)
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambd)(var))
    return var

In [14]:
#hyper parameters
batch_size = 32
data_size = dev_X.shape[0]
layer_dimension = [9,32,16,8,1]
n_layers = len(layer_dimension)
epoch = 300
#dev_ratio = 0.1

In [15]:
x = tf.placeholder(dtype=tf.float32, shape=(None, 9),name='x')
y_ = tf.placeholder(dtype=tf.float32, shape=(None, 1),name='y_')

In [16]:
cur_layer = x
in_dimension = layer_dimension[0]
for i in range(1, n_layers - 1):
    out_dimension = layer_dimension[i]
    w = get_weight([in_dimension, out_dimension], 0.01)
    b = tf.Variable(tf.zeros(out_dimension,dtype=tf.float32),dtype=tf.float32)
    A = tf.matmul(cur_layer, w) + b
    Z = tf.nn.relu(A)
    
    cur_layer = Z
    in_dimension = layer_dimension[i]

print(layer_dimension[n_layers - 2], layer_dimension[n_layers - 1])
w = get_weight([layer_dimension[n_layers - 2], layer_dimension[n_layers - 1]], 0.006)
print(w)
b = tf.Variable(tf.zeros(layer_dimension[n_layers - 1],dtype=tf.float32),dtype=tf.float32)
print(b)
# A_last = tf.matmul(cur_layer, w) + b
# y = tf.sigmoid(A_last)
# cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y,1e-4,1.0)) + (1 - y_) * tf.log(1 - tf.clip_by_value(y,1e-4,1.0)))
y = tf.matmul(cur_layer, w) + b
rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y, y_))))
tf.add_to_collection('losses', rmse)

Instructions for updating:
Use the retry module or similar alternatives.
8 1
<tf.Variable 'Variable_6:0' shape=(8, 1) dtype=float32_ref>
<tf.Variable 'Variable_7:0' shape=(1,) dtype=float32_ref>


In [17]:
with tf.name_scope('loss'):
    loss = tf.add_n(tf.get_collection('losses'))
    tf.summary.scalar('loss',loss)
global_ = tf.Variable(0)
add_global = global_.assign_add(1)
learning_rate =  tf.train.exponential_decay(1e-8, global_, 100, 0.9, staircase=True) #set learning rate decay
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [18]:
sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)
merged_summary_op = tf.summary.merge_all()
writer = tf.summary.FileWriter("logs/minst_log", sess.graph)
for i in range(epoch):
    for j in range(math.ceil(float(data_size / batch_size))):
        start = j * batch_size
        end = min(start + batch_size, data_size)

        batch_x = val_X[start:end, :] 
        #print(batch_x.shape)
        batch_y = val_y[start:end, :]
        #print(batch_y.shape)    
        rs, _, loss_value = sess.run([merged_summary_op, train_step, loss], feed_dict={x : batch_x, y_ : batch_y, global_ : i})
        if j % 1000 == 0:
            print('%d loss %f: ' % (j, loss_value))
            
    total_rmse_val = sess.run(rmse, feed_dict={x : val_X, y_ : val_y})
    total_rmse_dev = sess.run(rmse, feed_dict={x : dev_X, y_ : dev_y})
    print('%d epoch : train loss is %g ; dev loss is %g' % (i, total_rmse_val, total_rmse_dev))
    writer.add_summary(rs, i)

sess.close()

0 loss 1.927362: 
1000 loss 2.342706: 
2000 loss 2.064897: 
3000 loss 2.375466: 
4000 loss 1.759552: 
5000 loss 4.088017: 
6000 loss 2.696668: 
7000 loss nan: 
8000 loss nan: 
9000 loss nan: 
10000 loss nan: 
11000 loss nan: 
12000 loss nan: 
13000 loss nan: 
14000 loss nan: 
15000 loss nan: 
16000 loss nan: 
17000 loss nan: 
18000 loss nan: 
19000 loss nan: 
20000 loss nan: 
21000 loss nan: 
22000 loss nan: 
23000 loss nan: 
24000 loss nan: 
25000 loss nan: 
26000 loss nan: 
27000 loss nan: 
28000 loss nan: 
29000 loss nan: 
30000 loss nan: 
31000 loss nan: 
32000 loss nan: 
33000 loss nan: 


KeyboardInterrupt: 

In [12]:
sess.close()